In [ ]:
!pip install feedparser
!pip install GoogleNews
!pip install gnewsclient
!pip install snscrape==0.6.2.20230320
!pip install rake_nltk
!pip install geopy
!pip install pgeocode
!pip install requests lxml
!pip install beautifulsoup4
!pip install pretty-html-table
!pip install pyshorteners
!pip install xlsxwriter


In [ ]:
import pandas as pd
import requests
from GoogleNews import GoogleNews
from gnewsclient import gnewsclient
import snscrape.modules.twitter as sntwitter
from datetime import datetime, date, timedelta
import json
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from rake_nltk import Rake
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

# NEW: RSS-based fetching & helpers
import feedparser
from urllib.parse import quote_plus
import time
from datetime import datetime as dt


In [ ]:
def http_get_with_retries(session, url, headers=None, tries=3, backoff=2.0, timeout=30):
    """GET with simple exponential backoff; returns Response or raises last error."""
    last_exc = None
    for i in range(tries):
        try:
            r = session.get(url, headers=headers or {}, timeout=timeout, allow_redirects=True)
            if r.status_code == 200 and r.content:
                return r
            else:
                print(f"[HTTP] status={r.status_code} len={len(r.content)} url={url}")
        except Exception as e:
            last_exc = e
            print(f"[HTTP-ERR] try={i+1}/{tries} url={url} err={e}")
        time.sleep(backoff ** i)
    if last_exc:
        raise last_exc
    return None


In [ ]:
start_date = []
end_date = []
sources = ["bbc-news", "the-telegraph", "the-guardian-uk", "cnn", "abc-news-au",
           "dailymail.co.uk", "metro.co.uk", "mirror.co.uk", "news.google.com"]
all_keywords = ['strike', 'holiday', 'lockdown', 'inflation', 'grocery sales', 'carnival', 'festival', 'party',
                'Walmart', 'Tesco', "Sainsbury's", 'supply chain', 'flood', 'wendys', 'lidl']
keywords = ['Lidl','Waitrose','Tesco','Walmart','Sainsbury\'s','Aldi','Asda','Marks & Spencers','Morrison\'s']
events = ['autumn bank holiday']
all_events = ['autumn bank holiday']
final_prod_events = pd.DataFrame()
counter = 6000
gnews_client_topics = ['Top Stories','World','Nation','Business','Technology','Entertainment','Sports','Science','Health']


In [ ]:
# ---------------------------------------------
# COMPACT BRANCH DICTIONARY (replace with yours)
# ---------------------------------------------
branch_keyword_bu_num = {
    'Brighton': 114,
    'Dorchester': 120,
    'Esher': 121,
}
all_branch_keyword = list(branch_keyword_bu_num.keys())
branch_keyword = all_branch_keyword
countries = [['London']]
final = []
status_val = []


In [ ]:
# ---------- Text processing helpers ----------
def _removeNonAscii(s):
    return "".join(i for i in s if ord(i) < 128)

def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = text.replace('(ap)', '')
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'\\W+', ' ', text)
    text = re.sub(r'\\s+', ' ', text)
    text = re.sub(r"\\\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r'\"', '', text)
    text = re.sub('[^a-zA-Z ?!]+', '', text)
    text = _removeNonAscii(text)
    text = text.strip()
    return text

def remove_stopwords(word_tokens):
    filtered_sentence = []
    stop_words = stopwords.words('english')
    specific_words_list = ['char', 'u', 'hindustan', 'doj', 'washington']
    stop_words.extend(specific_words_list)
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])

tokenizer = RegexpTokenizer(r'\\w+')

def tokenize(x):
    return tokenizer.tokenize(x)

nltk.download('all')


In [ ]:
# ---------- Sentiment ----------
def sentiment_analysis(prod):
    if prod.empty:
        return prod.copy()
    prod['combined_text'] = prod['title'].map(str)
    prod['combined_text'] = prod['combined_text'].map(clean_text)
    prod['tokens'] = prod['combined_text'].map(tokenize)
    prod['tokens'] = prod['tokens'].map(remove_stopwords)
    prod['lems'] = prod['tokens'].map(lemmatize)
    sia = SIA()
    results = []
    for line in prod['lems']:
        pol_score = sia.polarity_scores(line)
        pol_score['lems'] = line
        results.append(pol_score)
    headlines_polarity = pd.DataFrame.from_records(results)
    headlines_polarity['label'] = 0
    headlines_polarity.loc[headlines_polarity['compound'] > 0.2, 'label'] = 1
    headlines_polarity.loc[headlines_polarity['compound'] < -0.2, 'label'] = -1
    headlines_polarity['word_count'] = headlines_polarity['lems'].apply(lambda x: len(str(x).split()))
    headlines_polarity = headlines_polarity.rename_axis(index=None)
    return pd.merge(prod, headlines_polarity, on=["lems"], how="left")


In [ ]:
# ---------- RSS-based fetch with fallbacks and logging ----------
def googleNewsByStreet():
    session = requests.Session()
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                       "(KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"),
        "Accept": "application/rss+xml,text/xml;q=0.9,*/*;q=0.8",
    }

    data = pd.DataFrame()

    def resolve_google_link(url: str) -> str:
        try:
            if "news.google.com" in url:
                r = http_get_with_retries(session, url, headers=headers, tries=2, backoff=2.0, timeout=20)
                return r.url if r is not None else url
        except Exception:
            pass
        return url

    total_pairs = 0
    nonempty_pairs = 0

    for branch in branch_keyword:
        for keyword in keywords:
            total_pairs += 1
            query = f"{branch} {keyword}"
            rss_24h = (
                f"https://news.google.com/rss/search?q={quote_plus(query)}+when:1d&hl=en-GB&gl=GB&ceid=GB:en"
            )
            rows = []
            try:
                resp = http_get_with_retries(session, rss_24h, headers=headers, tries=3, backoff=2.0, timeout=30)
                feed = feedparser.parse(resp.content if resp is not None else b"")
                for entry in feed.entries:
                    rows.append({
                        "title": entry.get("title", ""),
                        "link": resolve_google_link(entry.get("link", "")),
                        "desc": entry.get("summary", ""),
                        "datetime": entry.get("published", entry.get("updated", "")),
                        "media": getattr(getattr(entry, "source", None), "title", "")
                                 or (entry.get("source", {}) or {}).get("title", ""),
                        "keyword": keyword,
                        "branch": branch,
                        "bu_num": branch_keyword_bu_num.get(branch, None)
                    })
                if not rows:
                    rss_3d = (
                        f"https://news.google.com/rss/search?q={quote_plus(query)}+when:3d&hl=en-GB&gl=GB&ceid=GB:en"
                    )
                    print(f"[FALLBACK] 0 items for 24h; trying 3d for '{query}'")
                    resp2 = http_get_with_retries(session, rss_3d, headers=headers, tries=2, backoff=2.0, timeout=30)
                    feed2 = feedparser.parse(resp2.content if resp2 is not None else b"")
                    for entry in feed2.entries:
                        rows.append({
                            "title": entry.get("title", ""),
                            "link": resolve_google_link(entry.get("link", "")),
                            "desc": entry.get("summary", ""),
                            "datetime": entry.get("published", entry.get("updated", "")),
                            "media": getattr(getattr(entry, "source", None), "title", "")
                                     or (entry.get("source", {}) or {}).get("title", ""),
                            "keyword": keyword,
                            "branch": branch,
                            "bu_num": branch_keyword_bu_num.get(branch, None)
                        })
                if not rows:
                    konly = (
                        f"https://news.google.com/rss/search?q={quote_plus(keyword)}+when:1d&hl=en-GB&gl=GB&ceid=GB:en"
                    )
                    print(f"[FALLBACK] still 0; keyword-only 1d for '{keyword}'")
                    resp3 = http_get_with_retries(session, konly, headers=headers, tries=2, backoff=2.0, timeout=30)
                    feed3 = feedparser.parse(resp3.content if resp3 is not None else b"")
                    for entry in feed3.entries:
                        rows.append({
                            "title": entry.get("title", ""),
                            "link": resolve_google_link(entry.get("link", "")),
                            "desc": entry.get("summary", ""),
                            "datetime": entry.get("published", entry.get("updated", "")),
                            "media": getattr(getattr(entry, "source", None), "title", "")
                                     or (entry.get("source", {}) or {}).get("title", ""),
                            "keyword": keyword,
                            "branch": "",
                            "bu_num": None
                        })
            except Exception as e:
                print(f"[WARN] RSS fetch failed for '{query}': {e}")
            if rows:
                nonempty_pairs += 1
                df = pd.DataFrame(rows)
                data = pd.concat([data, df], ignore_index=True)
    data = data.drop(columns=["img", "site"], errors="ignore")
    print(f"[STATS] pairs={total_pairs} nonempty={nonempty_pairs} rows={len(data)} at {dt.utcnow().isoformat()}Z")
    try:
        data.to_csv("rss_debug_snapshot.csv", index=False)
    except Exception:
        pass
    final.append(data)


In [ ]:
def outsource_news():
    googleNewsByStreet()
    prod = pd.concat(final) if len(final) > 0 else pd.DataFrame()
    if prod.empty:
        print("[INFO] No RSS rows captured; continuing for email guard.")
        prod = pd.DataFrame(columns=["title","link","desc","datetime","media","keyword","branch","bu_num"]) 
    else:
        prod = prod.drop_duplicates('title', keep='first')
    print(prod)
    status_val.append(30)
    final_prod = sentiment_analysis(prod) if not prod.empty else prod.copy()
    final_prod = final_prod.replace(np.nan,'',regex=True)

    # Keyword sets (unchanged from your pipeline)
    store_keywords = ['opens','closes','closed','opened','open','close','shut','confining','unopen','opening',
                      'close down','closing','shut down','conclude','ending','shutdown','closedown','closure','temporary',
                      'extended','shutting','launch','shuts','closures']
    competitor_keywords = ['tesco','wendys','lidl','sainsburys','sainsbury','aldi','morrisons','spencer','asda',
                           'supermarket','co','ocado','sparks','b&m','iceland','waitrose']

    print(final_prod)
    if not final_prod.empty:
        for index, row in final_prod.iterrows():
            toks = row.get('tokens', [])
            if len(np.intersect1d(toks, store_keywords)) == 0:
                final_prod.drop(index=index, axis=0, inplace=True)
            else:
                if len(np.intersect1d(toks, competitor_keywords)) == 0:
                    final_prod.drop(index=index, axis=0, inplace=True)
        for index, row in final_prod.iterrows():
            for value in row.get('tokens', []):
                val = value.capitalize()
                if val in branch_keyword_bu_num:
                    final_prod.at[index,'bu_num'] = branch_keyword_bu_num[val]
                    final_prod.at[index,'branch'] = val
        final_prod = final_prod.drop_duplicates('title', keep='first')
        final_prod = final_prod.drop_duplicates('lems', keep='first')
        final_prod = final_prod.drop_duplicates('tokens', keep='first')
        final_prod['title'] = final_prod['title'].astype(str)
        final_prod['competitor_evt_indchar'] = ['Yes' if(len(np.intersect1d(x,competitor_keywords)) > 0) else 'No' for x in final_prod['tokens']]
        counter_guid = int(date.today().strftime("%Y%m%d"))
        final_prod['efsevt_guid'] = [(counter_guid*1000)+i for i in range(len(final_prod))]
        final_prod['guid'] = [(counter_guid*2000)+i for i in range(len(final_prod))]
        final_prod['fixed_annual_ind'] = 'n'
        final_prod['perm_env_ind'] = 'n'
        final_prod['cancelled_ind'] = 'n'
        final_prod['create_user'] = ''
        final_prod['update_user'] = ''
        final_prod['perm_env_ind'] = 'n'
        final_prod['crt_timestamp'] = date.today()
        final_prod['upd_timestamp'] = date.today()
        final_prod.rename(columns={'link':'source_of_event'}, inplace=True)
        final_prod[["datetime"]] = final_prod[["datetime"]].astype(str)
        final_prod.columns = final_prod.columns.str.upper()
        final_prod.to_csv('Events.csv', mode='a', index=False, header=False)
    else:
        final_prod = pd.DataFrame(columns=["TITLE","LINK","DESC","DATETIME","MEDIA","KEYWORD","BRANCH","BU_NUM"]).copy()
    return final_prod


In [ ]:
from geopy.geocoders import Photon, GoogleV3, Nominatim
import pgeocode
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return round(12742 * asin(sqrt(a)),2)

def place_distance(string1,string2):
    try:
        geolocator_addr = Nominatim(user_agent="efs")
        place = string1 + "," + string2
        place_2 = "Waitrose," + string2
        pin = geolocator_addr.geocode(place)
        pin_2 = geolocator_addr.geocode(place_2)
        print(pin)
        print(pin_2)
        print(pin.raw['lat'],pin.raw['lon'],pin_2.raw['lat'],pin_2.raw['lon'])
    except:
        return 'N/A'
    return distance(float(pin.raw['lat']),float(pin.raw['lon']),float(pin_2.raw['lat']),float(pin_2.raw['lon']))


In [ ]:
road = []
from bs4 import BeautifulSoup
import math

def distance_infrastructure(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return round(12742 * asin(sqrt(a)),2)

def place_distance_infrastructure(string2, work):
    temp = []
    try:
        route_df = pd.read_csv("Branch_Lat_Lon.csv")
        geolocator_addr = Nominatim(user_agent="http")
        place_2 = string2 + ", London, UK"
        pin_2 = geolocator_addr.geocode(place_2)
        print(pin_2)
        print(pin_2.raw['lat'],pin_2.raw['lon'])
        actual = 999999
        if(pin_2 != 'None'):
            for index, row in route_df.iterrows():
                dif = distance_infrastructure(float(pin_2.raw['lat']),float(pin_2.raw['lon']),row["lat"],row["lon"])
                if( dif < 20):
                    if(actual > dif):
                        actual = dif
                        branch_name = row["branch"]
            if(actual <= 1):
                print(actual)
                temp.append(branch_name)
                print(branch_name)
                print("Yes")
                road.append([ string2 + " " + work, branch_name, "", date.today(), branch_keyword_bu_num.get(branch_name,''), actual,""])
                return branch_name
        print(actual)
    except:
        return None

def infrastructure():
    url = "https://tfl.gov.uk/traffic/status/?Input=&lineIds=&dateTypeSelect=Future%20date&direction=&startDate="+date.today().strftime("%Y-%m-%d")+"T00%3A00%3A00&endDate="+date.today().strftime("%Y-%m-%d")+"T23%3A59%3A59&lat=51.50721740722656&lng=-0.12758620083332062&placeType=stoppoint&input=London%2C%20UK"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "lxml")
    ele = soup.select('div[class^="road-disruption"]')
    street = []
    works = []
    for element in ele:
        h2_tags = element.select('h4')
        p_tags = element.select('p[class^="topmargin"]')
        date_tags = element.select('p[class^="highlight dates"]')
        if date_tags:
            print(date_tags[0].text.strip("\n\n").split("\n"))
        for h2_tag,p_tag in zip(h2_tags,p_tags):
            if("Works" in p_tag.text):
                arr = h2_tag.text.strip().split(" ")
                word = ""
                for i in range(1,len(arr)-1):
                    if('(' not in arr[i]):
                        word = word + " " + arr[i]
                street.append(word)
                works.append(p_tag.text)
    street = list(set(street))
    print(len(street))
    for i,j in zip(street,works):
        place_distance_infrastructure(i,j)
    return road


In [ ]:
import smtplib, ssl
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table
from pyshorteners import Shortener
from io import BytesIO
from email.mime.application import MIMEApplication
from cryptography.fernet import Fernet

def mail_data(final_prod, final_prod_old):
    port = 465 # For SSL
    context = ssl.create_default_context()
    mail_df = pd.DataFrame()
    if final_prod.empty:
        mail_df = pd.DataFrame([{
            "TITLE": "No items captured",
            "BRANCH": "",
            "SOURCE": "",
            "DATETIME": date.today().strftime("%Y-%m-%d"),
            "BRANCH_NUM": "",
            "DISTANCE IN MILES": "",
            "LINK": ""
        }])
    else:
        mail_df["TITLE"] = final_prod["TITLE"]
        mail_df["BRANCH"] = final_prod["BRANCH"]
        mail_df["SOURCE"] = final_prod["MEDIA"]
        mail_df["DATETIME"] = final_prod["DATETIME"]
        mail_df["BRANCH_NUM"] = final_prod["BU_NUM"]
        distance_arr = []
        for index, row in final_prod.iterrows():
            if(row.get("KEYWORD") and row.get("BRANCH")):
                distance_arr.append(place_distance(row["KEYWORD"], row["BRANCH"]))
        if distance_arr:
            mail_df["DISTANCE IN MILES"] = distance_arr
        urls = []
        for index,row in final_prod.iterrows():
            try:
                x = Shortener().tinyurl.short(row.get("SOURCE_OF_EVENT",""))
            except Exception:
                x = row.get("SOURCE_OF_EVENT","")
            urls.append(x)
        mail_df["LINK"] = urls if urls else ""
        for index, row in mail_df.iterrows():
            if(row.get("DISTANCE IN MILES") != 'N/A'):
                if(isinstance(row.get("DISTANCE IN MILES"), (int, float)) and row["DISTANCE IN MILES"] > 25):
                    mail_df.drop(index=index, axis=0, inplace=True)
    all_prod = pd.concat([mail_df, final_prod_old]) if not mail_df.empty else mail_df
    road_list = infrastructure()
    print(road_list)
    road_df = pd.DataFrame(road_list, columns = ["TITLE", "BRANCH", "SOURCE", "DATETIME", "BRANCH_NUM", "DISTANCE IN MILES", "LINK"]) if road_list else pd.DataFrame(columns=["TITLE","BRANCH","SOURCE","DATETIME","BRANCH_NUM","DISTANCE IN MILES","LINK"]) 
    road_df = road_df.drop_duplicates("BRANCH", keep="first") if not road_df.empty else road_df
    html_table = mail_df.to_html(index=False, classes='example-table')
    road_table = road_df.to_html(index=False, classes='example-table')
    text = f'''Hello Alex and Tim,\n Herewith attaching the events captured for all the competitors (core event types) including all the branches from {(date.today() - timedelta(days = 1)).strftime('%d-%m-%Y')} to {date.today().strftime('%d-%m-%Y')} which are auto-generated from the script.\n\n\nThanks And Regards,\nSubhash\n\n\n\n'''
    html_table = html_table.replace('<th>', '<th style="padding: 10px 90px 10px 90px;">', 1)
    road_table = road_table.replace('<th>', '<th style="padding: 10px 80px 10px 80px;">', 1)
    html = f'''
<html>
<head>
 <style>
 table.example-table th{{ padding: 10px; text-align: center; background-color: #FFFFFF; font-weight: bold; font-size: 14px; width: 400px; }}
 table.example-table th:first-child {{ padding: 20px 100px 20px 100px; }}
 table.example-table td {{ padding: 5px; color: black; font-size: 12px; width: 400px; font-family: Century Gothic, sans-serif; }}
 </style>
</head>
<body>
 <pre>{text}</pre>
 {html_table}
 <br/><br/>
 {road_table}
</body>
</html>
'''
    part1 = MIMEText(html, 'html')
    msg = MIMEMultipart("alternative")
    msg['Subject'] = "Automated Event Capturing Model"
    recipients = ['subhash.verma@johnlewis.co.uk']
    msg['To'] = ", ".join(recipients)
    msg.attach(part1)
    file_name = date.today().strftime("%d-%m-%Y") + "_Events.xlsx"
    textStream = BytesIO()
    writer = pd.ExcelWriter(textStream, engine='xlsxwriter')
    all_prod.to_excel(writer,sheet_name="Competitor Events",index=False)
    road_df.to_excel(writer,sheet_name="Road Closure Events",index=False)
    writer.close()
    textStream.seek(0)
    attachment = MIMEApplication(textStream.read(), name= file_name)
    attachment['Content-Disposition'] = 'attachment; filename="{}"'.format(file_name)
    msg.attach(attachment)
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        dec = str(Fernet('egupkHT3QJHG1c5dcPGiWEZaWdH04_uhgyD-8lYNxWM=').decrypt(b'gAAAAABpHy2IRPVaNZJU3a2jDD68rGtj0jMYEvJyrWRJepy-wUXuHwKdmAzMTSDXAWkP4S8tUWCd6Q5egqHWKGFkMx18sIu6NUPerPx9TSkeFpCedLP3LAc='), 'UTF-8')
        print("IN")
        server.login("subhash.verma@johnlewis.co.uk", dec)
        server.sendmail("subhash.verma@johnlewis.co.uk", recipients, msg.as_string())
    return all_prod, road_df


In [ ]:
import gspread
if __name__ == '__main__':
    final_prod = outsource_news()
    final_prod_old = pd.DataFrame()
    all_prod, road_df = mail_data(final_prod, final_prod_old)
